In [1]:
import requests
import re
from bs4 import BeautifulSoup
import json
import urllib

In [2]:
# s = "https://yandex.ru/maps/239/sochi/stops/stop__9973362/"
city_code = '239'
city_name = 'sochi'
stop = 'stop__9973362'

s0 = 'href="/maps/'+str(city_code)+'/'+city_name+'/routes/'

s = 'https://yandex.ru/maps/{}/{}/stops/{}/'.format(city_code, city_name, stop)

In [3]:
url = s

In [4]:
req = requests.get(url)
bs = BeautifulSoup(req.text, 'html.parser')

In [5]:
lst_links = bs.find_all('a')

In [6]:
lst_routes_links = []
route_names = []
titles = []
for link in lst_links:
    if s0 in str(link) and link.text != 'Маршруты':
        lst_routes_links.append('https://yandex.ru'+link.get('href'))
        route_names.append(link.text)
        titles.append(link.get('title'))
# 

In [7]:
url1 = lst_routes_links[0]

In [8]:
req = requests.get(url1)
bs = BeautifulSoup(req.text, 'html.parser')

In [9]:
scrpt = bs.find_all('script')[-2].text
js = json.loads(scrpt)

In [10]:
js['masstransitLine']['features'][0]['features'][-1]

{'coordinates': [39.773594992, 43.557614811],
 'id': 'stop__9973554',
 'name': 'Микрорайон Искра',
 'region': {'bounds': [[39.1261495, 43.261256799670214],
   [40.3199745, 43.908055799670215]],
  'hierarchy': [225, 10995, 239],
  'id': 239,
  'seoname': 'sochi'}}

In [11]:
bs.find_all('script')[-2].get("type") == "application/json"

True

In [14]:
all_scrpt = bs.find_all('script')
for scr in all_scrpt:
    if scr.get("type") == "application/json":
        print(all_scrpt.index(scr))
# 

12


In [16]:
str1 = 'остановка общественного транспорта'
str_qt = urllib.parse.quote(str1)

In [20]:
srch = "https://yandex.ru/maps/239/sochi/search/{}".format(str_qt)

In [21]:
req = requests.get(srch)
bs = BeautifulSoup(req.text, 'html.parser')

In [33]:
all_scrpt = bs.find_all("script")

In [34]:
all_scrpt = bs.find_all("script")
for scr in all_scrpt:
    if scr.get('type') == "application/json":
        print(all_scrpt.index(scr))

14


In [28]:
scr1 = all_scrpt[13].text
js2 = json.loads(scr1)

In [32]:
js2.keys()

dict_keys(['counters', 'hosts', 'locale', 'lang', 'tld', 'query', 'baseUrl', 'originalUrl', 'uatraits', 'cspNonce', 'origin', 'csrfToken', 'xscriptCsrfToken', 'isUserFromPortal', 'isUserFromGoogle', 'availableLangs', 'currencyCode', 'banner', 'staticHost', 'pageMode', 'bookmarks', 'clientErrors', 'searchResultsCount', 'enableTaxi', 'enableAvia', 'enableZapravkiPromo', 'iframeCounter', 'tnsCounter', 'cloudApi', 'blaBlaCarBanner', 'search', 'routes', 'feedback', 'font', 'advert', 'serviceName', 'bilet', 'share', 'mobile', 'servicePanel', 'apiManual', 'mapMargin', 'business', 'troubleshooting', 'passport', 'morda', 'ruler', 'ugcCabinet', 'advertLandingUrl', 'advertLandingMetroUrl', 'becomeAuthorUrl', 'becomeOwnerUrl', 'agreementUrl', 'metroAgreementUrl', 'rightHoldersUrl', 'ymapsUrlTemplate', 'mrcLandingUrl', 'reviews', 'enableRoutesB2B', 'routesB2BUrl', 'enableMobileDistribution', 'bannerLayout', 'pharmacyLinks', 'mapLocation', 'mapType', 'userRegion', 'experiments', 'layerLinks', 'layer

In [34]:
js2['searchPreloadedResults'].keys()

dict_keys(['requestId', 'requestSerpId', 'requestContext', 'requestQuery', 'requestCorrectedQuery', 'requestBounds', 'displayType', 'totalResultCount', 'requestResults', 'requestSkip', 'items', 'bounds', 'categories', 'filters', 'hasLavkaBanner'])

In [48]:
js2['searchPreloadedResults']['items'][1].keys()

dict_keys(['type', 'requestId', 'analyticsId', 'title', 'description', 'address', 'coordinates', 'displayCoordinates', 'bounds', 'logId', 'uri', 'id', 'metro', 'stops', 'panorama', 'shortTitle', 'fullAddress', 'country', 'status', 'businessLinks', 'ratingData', 'sources', 'categories', 'businessProperties', 'modularSnippet', 'seoname', 'geoId', 'compositeAddress', 'references', 'subtitleItems', 'region', 'breadcrumbs'])

In [53]:
js2['searchPreloadedResults']['items'][1]['description']

'Россия, Краснодарский край, Сочи, улица Конституции СССР'

In [80]:
str_ct = 'Самара'
str_qt = urllib.parse.quote(str_ct)
srch_ct = "https://yandex.ru/maps/?text={}".format(str_qt)

In [81]:
req = requests.get(srch_ct)
bs = BeautifulSoup(req.text, 'html.parser')

In [83]:
alla = bs.find_all('a')
for aa in alla:
    cls = aa.get("class")
    try:
        if 'catalog-entry-point' in cls:
            hrf = aa.get("href")
            break
    except:
        pass
# 
try:
    city_id,city_name = hrf.split("maps/")[1].split("/catalog")[0].split("/")
except:
    city_id,city_name = None,None
    print("Error. Not found")
    pass
# 

In [85]:
lnk_tp = "https://yandex.ru/maps/"+ str(city_id) + "/" + str(city_name) + "/transport"

In [96]:
def find_js(lnk):
    req = requests.get(lnk)
    bs = BeautifulSoup(req.text, 'html.parser')
    all_scrpt = bs.find_all("script")
    for scr in all_scrpt:
        if scr.get('type') == "application/json":
            sc_ind = all_scrpt.index(scr)
    # 
    scrp_txt = all_scrpt[sc_ind].text
    js = json.loads(scrp_txt)
    
    return js

In [105]:
js_tp = find_js(lnk_tp)

In [245]:
def foo_apnd(lst,data):
    if data not in lst:
        lst.append(data)
    return lst

lst_lineid = []
lst_tpts = []
lst_names = []
i=0
for i in range(len(js_tp['vehicles'])):
    try:
        meta_veh = js_tp['vehicles'][i]['properties']['VehicleMetaData']['Transport']
        lineid = meta_veh['lineId']
        lst_lineid = foo_apnd(lst_lineid,lineid)
        try:
            transp_type = meta_veh['type']
            name_rt = meta_veh['name']
            lst_tpts = foo_apnd(lst_tpts,transp_type)
            lst_names = foo_apnd(lst_names,name_rt)
        except:
            lst_tpts = foo_apnd(lst_tpts,None)
            lst_names = foo_apnd(lst_names,None)

        # 
    except:
        pass
# 

In [248]:
lst_tpts

['minibus', 'bus', 'tramway', 'trolleybus']

In [ ]:
# dct_lnks = {}
# i=0
# # for i in range(len(lst_lineid)):
# for i in range(5):
#     lnkid = lst_lineid[i]
#     lnk_rt = 'https://yandex.ru/maps/{}/{}/routes/{}'.format(city_id, city_name, lnkid)
#     js_rt = find_js(lnk_rt)
#     dct_lnks[lnkid] = js_rt
# # 

In [235]:
i=0
js_tp['vehicles'][i]['properties']['VehicleMetaData']['Transport']#['name']

{'id': 'samara|127000',
 'threadId': '2193179214',
 'lineId': '2193179075',
 'name': '99',
 'Types': ['minibus'],
 'type': 'minibus',
 'uri': 'ymapsbm1://transit/line?id=2193179075&ll=50.206932%2C53.220111&name=99&r=6935&type=minibus',
 'seoname': '99'}

In [236]:
js_tp['vehicles'][i]['properties']['VehicleMetaData']['Transport']['name']

'99'

In [237]:
js_tp['vehicles'][i]['properties']['VehicleMetaData']['Transport']['type']

'minibus'

In [ ]:
# i=0
# lnk_rt = 'https://yandex.ru/maps/{}/{}/routes/{}'.format(city_id, city_name, lst_lineid[i])

In [108]:
i=0
lnkid = lst_lineid[i]
lnk_rt = 'https://yandex.ru/maps/{}/{}/routes/{}'.format(city_id, city_name, lnkid)
js = find_js(lnk_rt)

In [117]:
lst_ks = list(js.keys())

In [119]:
lst_ks[0]

'counters'

In [213]:
i=78

In [227]:
js['meta']['breadcrumbs'][3]#.keys()#['name']

{'name': 'Маршрутка 99',
 'type': 'search',
 'url': 'https://yandex.ru/maps/51/samara/routes/minibus_99/796d617073626d313a2f2f7472616e7369742f6c696e653f69643d32313933313739303735266c6c3d35302e32303639333225324335332e323230313131266e616d653d393926723d3639333526747970653d6d696e69627573/',
 'currentPage': True}

In [214]:
# i+=1
js[lst_ks[i]]

{'title': 'Маршрутка 99: маршрут, остановки и расписание в Самаре',
 'breadcrumbs': [{'name': 'Карты',
   'type': 'root',
   'url': 'https://yandex.ru/maps/'},
  {'name': 'Самара',
   'type': 'region',
   'url': 'https://yandex.ru/maps/51/samara/',
   'region': {'center': [50.101783, 53.195538], 'zoom': 12}},
  {'name': 'Общественный транспорт',
   'type': 'masstransit-home',
   'url': 'https://yandex.ru/maps/51/samara/transport/'},
  {'name': 'Маршрутка 99',
   'type': 'search',
   'url': 'https://yandex.ru/maps/51/samara/routes/minibus_99/796d617073626d313a2f2f7472616e7369742f6c696e653f69643d32313933313739303735266c6c3d35302e32303639333225324335332e323230313131266e616d653d393926723d3639333526747970653d6d696e69627573/',
   'currentPage': True}]}

In [212]:
i

79

In [210]:
lst_ks[i]

'meta'

In [79]:
js['masstransitLine']['features'][1]

{'features': [{'id': 'stop__10001445',
   'coordinates': [50.098717449, 53.186512972],
   'name': 'Улица Высоцкого',
   'region': {'id': 51,
    'hierarchy': [225, 11131, 51],
    'seoname': 'samara',
    'bounds': [[49.772227, 52.96536237206464],
     [50.431338999999994, 53.42448437206464]]}},
  {'points': [[50.098645, 53.186561999999995],
    [50.098779, 53.186758999999995],
    [50.098856, 53.186871],
    [50.098904999999995, 53.186941999999995],
    [50.099044, 53.187145],
    [50.099087, 53.187208],
    [50.099309, 53.187535999999994],
    [50.099329999999995, 53.187566],
    [50.099418, 53.187695999999995],
    [50.099491, 53.187802999999995],
    [50.099647, 53.188032],
    [50.099705, 53.188119],
    [50.099776999999996, 53.188221999999996],
    [50.100004999999996, 53.188556999999996],
    [50.100153999999996, 53.188776999999995],
    [50.100272, 53.188952],
    [50.100516999999996, 53.189311],
    [50.100654, 53.189510999999996],
    [50.100874, 53.189834],
    [50.101006999

In [64]:
js['vehicles'][0]['properties']['VehicleMetaData']['Transport']['lineId']

'2193179125'

In [65]:
lineid = '2193179125'

In [69]:
l3 = 'https://yandex.ru/maps/{}/{}/routes/{}'.format(city_id, city_name, lineid)

In [68]:
print(s)

https://yandex.ru/maps/51/samara/routes/2193179125


In [43]:
l2 = l1 + "/buses"

In [44]:
print(l2)

https://yandex.ru/maps/51/samara/transport/buses


In [ ]:
# "https://yandex.ru/maps/51/samara/transport/buses/?l=masstransit&ll=50.212459%2C53.227108&z=11"

In [ ]:
# https://yandex.ru/maps/51/samara/transport/?ll=50.183735%2C53.233963&z=10